In [1]:
import json
data = {}
data["truck_types"] = [
    {
        'type': 'BigTruck',
        'speed': 45,
        'fuel_consumption': 0.25,
        'fuel_capacity': 300,
        'capacities': {'mixed': 35000},
        'unloading_time': 0.05
    }
]
data["container_types"] = [
    {
        'type': 'RegularContainer',
        'capacity': 1100,
        'garbage_type': 'mixed',
        'processing_time': 0.05
    }
]

data["garbage_types"] = ["mixed"]


with open("lviv_config.json", "w") as f:
    f.write(json.dumps(data, indent=4))

In [2]:
def create_input(original_input, output, trucks_count):

    with open(original_input) as f:
        data = json.load(f)
    for cont in data["containers"]:
        cont["type"] = "RegularContainer"
    truck = data["trucks"][0]
    truck["type"] = "BigTruck"
    data["trucks"] = [truck] * trucks_count
    data["landfills"][0]["latitude"] = 49.9012481;
    data["landfills"][0]["longitude"] = 24.0300928;
    with open(output, "w") as f:
        f.write(json.dumps(data, indent=4))


In [3]:
import subprocess
import sys
import os
import time


PROGRAM_PATH = "./routing-problem"


def create_dir(path):
    if os.path.exists(path):
        return

    os.makedirs(path)


def run(params):
    args = [f"--{name}={value}" for name, value in params.items()]
    command = f"{PROGRAM_PATH} {' '.join(args)}"
    res = os.system(command)
    if res != 0:
        print("ERROR")
        print("Command:", command)
        print("RES:", res)
        
def generate_params(search_data):
    keys = []
    all_params = [] 
    for key in search_data:
        if type(search_data[key]) == list:
            all_params.append(search_data[key])
        else:
            all_params.append([search_data[key]])
        keys.append(key)
    finished = False
    
    indices = [0] * len(keys)
    while not finished:
        params = {keys[i]: all_params[i][indices[i]] for i in range(len(keys))}
        yield params
        for i in range(len(keys)):
            index = len(keys) - i - 1
            if len(all_params[index]) - 1 == indices[index]:
                indices[index] = 0
                if index == 0:
                    finished = True
                    break
            else:
                indices[index] += 1
                break
                    
        
def perform_becnhmarks(searches_data, base_folder, filename, config, original_file="data/Lviv/input.json"):
    create_dir(base_folder)
    i = 0
    for search_data in searches_data:
        for params in generate_params(search_data):
            for key, value in params.items():
                if key == "output" or key == "input":
                    continue
                print(f"{key}={value},", end=" ")
                if key == "trucks_count":           
                    create_input(original_file, filename, value)
            print(f"\nIteration: {i}")
            if i < 150:
                i += 1
                continue
            del params["trucks_count"]
            params["config"]  = config
            params["output"] = f"{base_folder}/output_{i}.json"
            params["input"] = filename
            params["real"] = True
            start_time = time.time()
            run(params)
            execution_time = time.time() - start_time
            with open(params["output"]) as f:
                output = json.load(f)
            for param in params:
                output[param] = params[param]
            output["execution_time"] = execution_time
            print("Execution time", execution_time)
            json_string = json.dumps(output, indent=4)
            with open(params["output"], "w") as f:
                f.write(json_string)
            i += 1

In [4]:
heuristics = [
    "MOVE_IN_EXPORTATION",
    "REMOVE_LOOPS_SWAP_ANY_EXPORTATION",
    "REMOVE_LOOPS_MOVE_EXPORTATION",
    "ANY_EXPORTATION_PROB1",
    "ANY_MOVE_EXPORTATION_FIXED2",
    "ANY_EXPORTATION_TIME1",
    "ANY_EXPORTATION",
    "ANY_MOVE_EXPORTATION",
    "ANY_MOVE_SWAP",
]

swap_any_heuristics = [
    "ANY_MOVE_SWAP",
    "ANY_EXPORTATION_PROB1",
    "REMOVE_LOOPS_SWAP_ANY_EXPORTATION",
    "ANY_MOVE_EXPORTATION_FIXED2",
    "ANY_EXPORTATION_TIME1",
    "ANY_EXPORTATION",
    "ANY_MOVE_EXPORTATION",
]

searches = [
    {
        "search": "NO_SEARCH",
        "base_solution": ["CLUSTER", "SIMPLE_GREEDY"],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "TABU",
        "heuristic": heuristics,
        "base_solution": ["SIMPLE_GREEDY"],
        "tabu_queue_coef": [0.3, 0.5],
        "iterations": [200, 500],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics[:3],
        "base_solution":  ["SIMPLE_GREEDY"],
        "temp_coef": [0.00001, 0.0001],
        "iterations": [3000, 5000],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "TABU",
        "heuristic": swap_any_heuristics,
        "base_solution": ["SIMPLE_GREEDY"],
        "tabu_queue_coef": [0.3],
        "iterations": [500],
        "trucks_count": [10, 15, 20],
        "swap_any_to_generate": [500]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics[3:],
        "base_solution":  ["SIMPLE_GREEDY"],
        "temp_coef": [0.00001, 0.0001],
        "iterations": [3000, 5000],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "GENETIC",
        "heuristic": heuristics,
        "base_solution": ["SIMPLE_GREEDY"],
        "mutation_probability": [0.1, 0.3],
        "iterations": [200, 400],
        "population_size": [10, 20],
        "trucks_count": [10, 15, 20]
    },
        {
        "search": "TABU",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER"],
        "tabu_queue_coef": [0.3, 0.5],
        "iterations": [200, 500],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER"],
        "temp_coef": [0.00001, 0.0001],
        "iterations": [3000, 5000],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "GENETIC",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER"],
        "mutation_probability": [0.1, 0.3],
        "iterations": [200, 400],
        "population_size": [10, 20],
        "trucks_count": [10, 15, 20]
    },
    
]


perform_becnhmarks(searches, "results_lviv", "lviv_input.json", "lviv_config.json")

search=NO_SEARCH, base_solution=CLUSTER, trucks_count=10, 
Iteration: 0
search=NO_SEARCH, base_solution=CLUSTER, trucks_count=15, 
Iteration: 1
search=NO_SEARCH, base_solution=CLUSTER, trucks_count=20, 
Iteration: 2
search=NO_SEARCH, base_solution=SIMPLE_GREEDY, trucks_count=10, 
Iteration: 3
search=NO_SEARCH, base_solution=SIMPLE_GREEDY, trucks_count=15, 
Iteration: 4
search=NO_SEARCH, base_solution=SIMPLE_GREEDY, trucks_count=20, 
Iteration: 5
search=TABU, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=200, trucks_count=10, 
Iteration: 6
search=TABU, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=200, trucks_count=15, 
Iteration: 7
search=TABU, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=200, trucks_count=20, 
Iteration: 8
search=TABU, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=500, trucks_count=10, 

search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=200, trucks_count=20, 
Iteration: 68
search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=500, trucks_count=10, 
Iteration: 69
search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=500, trucks_count=15, 
Iteration: 70
search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=500, trucks_count=20, 
Iteration: 71
search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.5, iterations=200, trucks_count=10, 
Iteration: 72
search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.5, iterations=200, trucks_count=15, 
Iteration: 73
search=TABU, heuristic=ANY_EXPORTATION_TIME1, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.5, iterations=200, trucks_count=20, 
Iterat

search=ANNEALING, heuristic=REMOVE_LOOPS_SWAP_ANY_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=0.0001, iterations=5000, trucks_count=20, 
Iteration: 137
search=ANNEALING, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=3000, trucks_count=10, 
Iteration: 138
search=ANNEALING, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=3000, trucks_count=15, 
Iteration: 139
search=ANNEALING, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=3000, trucks_count=20, 
Iteration: 140
search=ANNEALING, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=5000, trucks_count=10, 
Iteration: 141
search=ANNEALING, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=5000, trucks_count=15, 
Iteration: 142
search=ANNEALING, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_s

1812.97
Execution time 133.94250917434692
search=ANNEALING, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, temp_coef=0.0001, iterations=3000, trucks_count=20, 
Iteration: 179
1842.08
1842.08
Execution time 134.93960809707642
search=ANNEALING, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, temp_coef=0.0001, iterations=5000, trucks_count=10, 
Iteration: 180
1779.33
1779.33
Execution time 187.50112009048462
search=ANNEALING, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, temp_coef=0.0001, iterations=5000, trucks_count=15, 
Iteration: 181
1812.97
1812.97
Execution time 182.3381519317627
search=ANNEALING, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, temp_coef=0.0001, iterations=5000, trucks_count=20, 
Iteration: 182
1842.08
1842.08
Execution time 176.28110694885254
search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION_FIXED2, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=3000, trucks_count=10, 
Iteration: 183
1779.33
1779.3

1812.97
1773.93
Execution time 302.66387486457825
search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=3000, trucks_count=20, 
Iteration: 221
1842.08
1792.15
Execution time 324.8714048862457
search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=5000, trucks_count=10, 
Iteration: 222
1779.33
1723.34
Execution time 489.91157126426697
search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=5000, trucks_count=15, 
Iteration: 223
1812.97
1766.25
Execution time 480.8856120109558
search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=5000, trucks_count=20, 
Iteration: 224
1842.08
1791.54
Execution time 453.0097110271454
search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=0.0001, iterations=3000, trucks_count=10, 
Iteration: 225
1779.33
1758.99
Execut

1842.08
1804.9
Execution time 248.1870539188385
search=GENETIC, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=10, trucks_count=10, 
Iteration: 261
1779.33
1740.58
Execution time 344.0075557231903
search=GENETIC, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=10, trucks_count=15, 
Iteration: 262
1812.97
1780.69
Execution time 374.3185350894928
search=GENETIC, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=10, trucks_count=20, 
Iteration: 263
1842.08
1804.96
Execution time 359.6955780982971
search=GENETIC, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=20, trucks_count=10, 
Iteration: 264
1779.33
1739.44
Execution time 661.4772260189056
search=GENETIC, heuristic=MOVE_IN_EXPORTATION, base_solution=SIMPLE_GREE

1842.08
1801.67
Execution time 322.30875992774963
search=GENETIC, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.1, iterations=400, population_size=10, trucks_count=10, 
Iteration: 297
1779.33
1737.78
Execution time 436.18774819374084
search=GENETIC, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.1, iterations=400, population_size=10, trucks_count=15, 
Iteration: 298
1812.97
1775.85
Execution time 474.5372591018677
search=GENETIC, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.1, iterations=400, population_size=10, trucks_count=20, 
Iteration: 299
1842.08
1801.12
Execution time 425.2722499370575
search=GENETIC, heuristic=REMOVE_LOOPS_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, mutation_probability=0.1, iterations=400, population_size=20, trucks_count=10, 
Iteration: 300
1779.33
1734.33
Execution time 872.8776021003723
search=GENETIC, heuristic=REM

1779.33
1741.1
Execution time 267.30071687698364
search=GENETIC, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=10, trucks_count=15, 
Iteration: 334
1812.97
1778.3
Execution time 320.92781615257263
search=GENETIC, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=10, trucks_count=20, 
Iteration: 335
1842.08
1807.37
Execution time 303.923898935318
search=GENETIC, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=20, trucks_count=10, 
Iteration: 336
1779.33
1745.22
Execution time 473.32813119888306
search=GENETIC, heuristic=ANY_EXPORTATION_PROB1, base_solution=SIMPLE_GREEDY, mutation_probability=0.3, iterations=400, population_size=20, trucks_count=15, 
Iteration: 337
1812.97
1780.6
Execution time 463.22724413871765
search=GENETIC, heuristic=ANY_EXPORTATION_PROB1, base_solution=

FileNotFoundError: [Errno 2] No such file or directory: 'results_lviv/output_350.json'

In [22]:
output["config"] = input_data["config"]

In [ ]:
start = time.time()
executionTime = time.time() - start
print(executionTime)

In [28]:
import pandas as pd
df = pd.DataFrame(data=df_data)


In [31]:
df

,trucks_count,heuristic,base_solution,cluster_coef,search,iterations,tabu_queue_coef,execution_time,total_distance,total_fuel,total_time,orphans_count,unused_trucks
0,10,SWAP_IN_EXPORTATION,CLUSTER,2.0,TABU,100,0.5,1764.343583,1769.3647,442.341175,11.930107,0,0
1,10,SWAP_NEIGHBOURS,CLUSTER,2.0,TABU,100,0.5,142.279817,1792.8342,448.208550,12.342329,0,0
2,10,MOVE_IN_EXPORTATION,CLUSTER,2.0,TABU,100,0.5,2112.738208,1753.2722,438.318050,11.864027,0,0
3,10,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,CLUSTER,2.0,TABU,100,0.5,780.696362,1780.0472,445.011800,12.037716,0,0
4,10,REMOVE_LOOPS_MOVE_EXPORTATION,CLUSTER,2.0,TABU,100,0.5,953.915394,1754.1111,438.527775,11.845513,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,30,ANY_EXPORTATION_TIME2,CLUSTER,2.0,TABU,200,0.5,490.789041,1940.8466,485.211650,2.973104,0,0
109,30,ANY_EXPORTATION,CLUSTER,2.0,TABU,200,0.5,1307.987685,1913.8733,478.468325,2.961662,0,0
110,30,ANY_MOVE_EXPORTATION,CLUSTER,2.0,TABU,200,0.5,1845.931006,1897.8070,474.451750,2.939920,0,0
111,30,ANY_MOVE_SWAP,CLUSTER,2.0,TABU,200,0.5,647.187127,1930.2964,482.574100,2.999896,0,0


In [37]:
row = df[df["trucks_count"] == 10].nsmallest(1, "total_distance")
print(f"{row['search'].values[0]},"
      f"{row['base_solution'].values[0]},"
      f" {row['heuristic'].values[0]}, Dist: {row['total_distance'].values[0]} {row['iterations'].values[0]}")
row = df[df["trucks_count"] == 10].nlargest(1, "total_distance")
print(f"{row['search'].values[0]},"
      f"{row['base_solution'].values[0]},"
      f" {row['heuristic'].values[0]}, Dist: {row['total_distance'].values[0]} {row['iterations'].values[0]}")

TABU,CLUSTER, REMOVE_LOOPS_MOVE_EXPORTATION, Dist: 1746.1165 200
TABU,CLUSTER, ANY_EXPORTATION_FIXED2, Dist: 1794.8908999999999 100


In [30]:
df.to_csv("lviv_results.csv")

In [ ]:
with open("output.json") as f:
    data = json.load(f)
data

In [ ]:
total_distance = 0
for route in data["Routes"]:
    for i in range(1, len(route["Points"])):
        total_distance += ((route["Points"][i]["Longitude"] - route["Points"][i-1]["Longitude"])**2 
                          +(route["Points"][i]["Latitude"] - route["Points"][i-1]["Latitude"])**2 ) ** 0.5
    print(total_distance)
    
total_distance

In [5]:
create_input("data/Lviv/input.json", "lviv_input.json", 10)

In [14]:
import time
start = time.time()
params = {
    "iterations": 100,
    "config": "lviv_config.json",
    "input": "lviv_input.json",
    "real": "true",
    "heuristic": "SWAP_NEIGHBOURS"
}
run(params)
print("Time:", time.time() - start)

1795.67
1794.08
Time: 116.96102499961853


In [22]:
import time
start = time.time()
params = {
    "iterations": 100,
    "config": "lviv_config.json",
    "input": "lviv_input.json",
    "real": "true",
    "heuristic": "SWAP_IN_EXPORTATION",
    "search": "NO_SEARCH",
}
run(params)
print("Time:", time.time() - start)

1795.67
1795.67
Time: 89.68361210823059
